In [ ]:
# VanDerPol

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt


# We use Python's `functools.partial` to create a new function based on `fast_vanderpol` 
# with "eps" set to zero.
from functools import partial
from scipy.integrate import odeint
from sympy import symbols, Eq, solve, Function, Matrix, diff
#from scipy.integrate import solve_ivp


# Define the ODE systems
def fast_vanderpol(y, t, eps):
    x, y = y
    dxdt = y - (1/3) * x**3 + x
    dydt = -eps * x
    return [dxdt, dydt]

# Create a new function, "fast_subsystem" with eps=0 based on fast_vanderpol
fast_subsystem = partial(fast_vanderpol, eps=0)

# Define the Jacobian function
def jacobian(func, y):
    n = len(y)
    J = np.zeros((n, n))
    for i in range(n):
        epsilon = 1e-6  # Small perturbation
        y_perturbed = np.array(y.copy(), dtype=float)  
        y_perturbed[i] += epsilon
        dy = np.array(func(y_perturbed, 0)) - np.array(func(y, 0))
        J[:, i] = dy / epsilon
    return J

# Find equilibrium points of fast_subsystem numerically
equilibrium_points = [np.zeros(len(fast_vanderpol([0, 0], 0, 0)))] # [np.zeros(len(y))]

# Compute Jacobian matrix at each equilibrium point
for eq_point in equilibrium_points:
    J_eq = jacobian(fast_subsystem, eq_point)
    eigenvalues, eigenvectors = np.linalg.eig(J_eq)
    print("Jacobian Matrix:")
    print(J_eq)
    print("Eigenvalues:")
    print(eigenvalues)
    print("Eigenvectors:")
    print(eigenvectors)
    print()
    # Check if any eigenvalue has a pure imaginary part
    if any(np.imag(eigenvalues) != 0):
        print("The system is NOT NORMALLY HYPERBOLIC. Stopping...")
        break
    else:
        print("The system is NORMALLY HYPERBOLIC. Proceeding...\n")


def slow_vanderpol(y, tau, eps):
    x, y = y
    dxdt =  (y - (1/3) * x**3 + x)/eps
    dydt = - x
    return [dxdtau, dydtau]

def slow_subsystem(y, tau):
    x, y = y
    dxdtau = x / (1 - x ** 2)
    dydtau = - x
    return [dxdtau, dydtau]


# Initial points x, y, z:
x_init, y_init = 1.0, 1.0
print('The initial points of the BVP:', f"x_init = {x_init}, y_init = {y_init}")

t_end = 100
t  =  np.linspace(0, t_end, 100)
t2  =  np.linspace(0, -t_end, 100)
eps = 0.01
tau = eps * t

# Define initial conditions over the slow manifold
x1, y1 = 2.1, 1.0
print('The starting points on the slow manifold:', f"x1 = {x1}, y1 = {y1}")

init_slow = [x1, y1]  
# Solve the differential equation using odeint
slow_subsystem_solution = odeint(slow_subsystem, init_slow, tau)
# Find the time at which x is closest to x=1
x_values = slow_subsystem_solution[:, 0]  # Extract x values from the solution
closest_index = np.argmin(np.abs(x_values - 1))  # Find index with minimum absolute difference
closest_time1 = tau[closest_index]  # Corresponding time value

print('The time at which x is closest to x=1:', closest_time1)

# Extract the last values of x, y on the slow_subsystem
x2, y2 = slow_subsystem_solution[closest_index]
print('The ending points on the slow manifold (piece1):', f"x2 = {x2}, y2 = {y2}")

x_end, y_end = -0.5578, 0.5
print('The ending points of the BVP:', f"x_end = {x_end}, y_end = {y_end}")


# solve ODEs
intit_bvp = [x_init  y_init]
x_real, y_real = odeint(fast_vanderpol, intit_bvp, t, args=(mu,)).T



# Exact solution
x_exact_fast, y_exact_fast, z_exact_fast = fast_solution(t, eps, x_init, y_init, z_init)


# Exact solution for the slow system using the final state of the fast system as initial conditions
x_exact_slow, y_exact_slow, z_exact_slow = slow_solution(tau, eps, x_init, y_init, z_init)
print(x_exact_slow[-1], y_exact_slow[-1], z_exact_slow[-1])


x_exact_fast2, y_exact_fast2, z_exact_fast2 = fast_solution(t2, eps, x_end, y_end, z_end)
print(x_exact_fast2[-1], y_exact_fast2[-1], z_exact_fast2[-1])

# Transform to tensor
t_tensor = torch.tensor(t.reshape(-1, 1), dtype=torch.float64) 
x_real_fast_tensor = torch.tensor(x_exact_fast.reshape(-1, 1), dtype=torch.float64)
y_real_fast_tensor = torch.tensor(y_exact_fast.reshape(-1, 1), dtype=torch.float64)
z_real_fast_tensor = torch.tensor(z_exact_fast.reshape(-1, 1), dtype=torch.float64)

tau_tensor = torch.tensor(tau.reshape(-1, 1), dtype=torch.float64)
x_real_slow_tensor = torch.tensor(x_exact_slow.reshape(-1, 1), dtype=torch.float64)
y_real_slow_tensor = torch.tensor(y_exact_slow.reshape(-1, 1), dtype=torch.float64)
z_real_slow_tensor = torch.tensor(z_exact_slow.reshape(-1, 1), dtype=torch.float64)

t2_tensor = torch.tensor(t2.reshape(-1, 1), dtype=torch.float64)
x_real_fast2_tensor = torch.tensor(x_exact_fast2.reshape(-1, 1), dtype=torch.float64)
y_real_fast2_tensor = torch.tensor(y_exact_fast2.reshape(-1, 1), dtype=torch.float64)
z_real_fast2_tensor = torch.tensor(z_exact_fast2.reshape(-1, 1), dtype=torch.float64)


def input_transform(t_tensor):
    return torch.cat([t_tensor], dim=1)


class fast_vanderpol_PINN(nn.Module):
    def __init__(self):
        super(fast_vanderpol_PINN, self).__init__()
        self.fc1 = nn.Linear(1, 50)
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, 2)

    def forward(self, t):
        x = F.tanh(self.fc1(t))
        x = F.tanh(self.fc2(x))
        x = self.fc3(x)
        return x

class slow_vanderpol_PINN(nn.Module):
    def __init__(self):
        super(slow_vanderpol_PINN, self).__init__()
        self.fc1 = nn.Linear(1, 50)
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, 2)

    def forward(self, tau):
        x = F.tanh(self.fc1(tau))
        x = F.tanh(self.fc2(x))
        x = self.fc3(x)
        return x
    
class fast_vanderpol_PINN2(nn.Module):
    def __init__(self):
        super(fast_vanderpol_PINN2, self).__init__()
        self.fc1 = nn.Linear(1, 50)
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, 2)

    def forward(self, t):
        x = F.tanh(self.fc1(t))
        x = F.tanh(self.fc2(x))
        x = self.fc3(x)
        return x



def loss_func_fast(model, t_tensor, x_exact_fast, y_exact_fast, z_exact_fast, x_init, y_init, z_init, x_T, y_T, z_T, eps, random_points=10):
    t_tensor.requires_grad = True
    pred_fast = model(t_tensor)
    x_pred_fast, y_pred_fast, z_pred_fast = pred_fast[:, 0].unsqueeze(1), pred_fast[:, 1].unsqueeze(1), pred_fast[:, 2].unsqueeze(1)
    # Compute residuals using torch.autograd.grad
    def compute_residuals_autograd(system, output, time, epsilon):
        derivatives = []
        for idx in range(len(output)):
            derivative = torch.autograd.grad(output[idx], time, grad_outputs=torch.ones_like(output[idx]), create_graph=True, retain_graph=True)[0]
            derivatives.append(derivative)
        residuals = [derivative - output[idx] for idx, derivative in enumerate(derivatives)]
        return residuals
    residual_fast_vanderpol_autograd = compute_residuals_autograd(fast_vanderpol, [x_pred_fast, y_pred_fast, z_pred_fast], t_tensor, eps)
    residual_fast_subsystem_autograd = compute_residuals_autograd(fast_subsystem, [x_pred_fast, y_pred_fast, z_pred_fast], t_tensor, 0)  # eps=0 for fast_subsystem
    
    physics_loss_fast = torch.mean(torch.stack(residual_fast_vanderpol_autograd)**2)  +\
                        torch.mean(torch.stack(residual_fast_subsystem_autograd)**2)
   
    init_loss_fast = torch.square(x_pred_fast[0] - x_init) + torch.square(y_pred_fast[0] - y_init) + torch.square(z_pred_fast[0] - z_init)
    boundary_loss_fast = torch.square(x_pred_fast[-1] - x_T) + torch.square(y_pred_fast[-1] - y_T) + torch.square(z_pred_fast[-1] - z_T)

    # Combine all loss terms
    total_loss_fast = physics_loss_fast + init_loss_fast + boundary_loss_fast 

    return total_loss_fast

def loss_func_slow(model, tau_tensor, x_exact_slow, y_exact_slow, z_exact_slow, x1, y1, z1, x_TT, y_TT, z_TT, eps, random_points=10):
    tau_tensor.requires_grad = True
    pred_slow = model(tau_tensor)
    x_pred_slow, y_pred_slow, z_pred_slow = pred_slow[:, 0].unsqueeze(1), pred_slow[:, 1].unsqueeze(1), pred_slow[:, 2].unsqueeze(1)
    ones = torch.ones_like(x_pred_slow, requires_grad=True)
    dx_dtau = torch.autograd.grad(x_pred_slow, tau_tensor, grad_outputs=ones, retain_graph=True, create_graph=True)[0]
    dy_dtau = torch.autograd.grad(y_pred_slow, tau_tensor, grad_outputs=ones, retain_graph=True, create_graph=True)[0]
    dz_dtau = torch.autograd.grad(z_pred_slow, tau_tensor, grad_outputs=ones, retain_graph=True, create_graph=True)[0]

    residual1_slow = 0  # dx_dtau + x_pred_slow / eps
    residual2_slow = 0  # dy_dtau - (2 * y_pred_slow + eps * x_pred_slow) / eps
    residual3_slow = dz_dtau - x_pred_slow**2 - 1
    init_loss_slow = torch.square(x_pred_slow[0] - x1) + torch.square(y_pred_slow[0] - y1) + torch.square(z_pred_slow[0] - z1)
    physics_loss_slow = torch.mean(residual1_slow**2 + residual2_slow**2 + residual3_slow**2)

    random_indices = torch.randint(0, x_exact_slow.shape[0], (random_points,))
    
    boundary_loss_slow = torch.square(x_pred_slow[-1] - x_TT) + torch.square(y_pred_slow[-1] - y_TT) + torch.square(z_pred_slow[-1] - z_TT)
    total_loss_slow = physics_loss_slow + init_loss_slow + boundary_loss_slow
    return total_loss_slow


def loss_func_fast(model, t2_tensor, x_exact_fast2, y_exact_fast2, z_exact_fast2, x_end, y_end, z_end, x2_T, y2_T, z2_T, eps, random_points=10):
    t2_tensor.requires_grad = True
    pred_fast2 = model(t2_tensor)
    x_pred_fast2, y_pred_fast2, z_pred_fast2 = pred_fast2[:, 0].unsqueeze(1), pred_fast2[:, 1].unsqueeze(1), pred_fast2[:, 2].unsqueeze(1)
    ones = torch.ones_like(x_pred_fast2, requires_grad=True)  
    dx_dt2 = torch.autograd.grad(x_pred_fast2, t2_tensor, grad_outputs=ones, retain_graph=True, create_graph=True)[0]
    dy_dt2 = torch.autograd.grad(y_pred_fast2, t2_tensor, grad_outputs=ones, retain_graph=True, create_graph=True)[0]
    dz_dt2 = torch.autograd.grad(z_pred_fast2, t2_tensor, grad_outputs=ones, retain_graph=True, create_graph=True)[0]

    # Compute residuals for fast_vanderpol
    residual1_fast2 = dx_dt2 - fast_vanderpol([x_pred_fast2, y_pred_fast2, z_pred_fast2], t2_tensor, eps)[0]
    residual2_fast2 = dy_dt2 - fast_vanderpol([x_pred_fast2, y_pred_fast2, z_pred_fast2], t2_tensor, eps)[1]
    residual3_fast2 = dz_dt2 - fast_vanderpol([x_pred_fast2, y_pred_fast2, z_pred_fast2], t2_tensor, eps)[2]

    # Compute residuals for fast_subsystem
    residual4_fast2 = dx_dt2 - fast_subsystem([x_pred_fast2, y_pred_fast2, z_pred_fast2], t2_tensor)[0]
    residual5_fast2 = dy_dt2 - fast_subsystem([x_pred_fast2, y_pred_fast2, z_pred_fast2], t2_tensor)[1]
    residual6_fast2 = dz_dt2 - fast_subsystem([x_pred_fast2, y_pred_fast2, z_pred_fast2], t2_tensor)[2]

    physics_loss_fast2 = torch.mean(residual1_fast2**2 + residual2_fast2**2 + residual3_fast2**2 +
                                    residual4_fast2**2 + residual5_fast2**2 + residual6_fast2**2)

    init_loss_fast2 = torch.square(x_pred_fast2[0] - x_end) + torch.square(y_pred_fast2[0] - y_end) + torch.square(z_pred_fast2[0] - z_end)
    boundary_loss_fast2 = torch.square(x_pred_fast2[-1] - x2_T) + torch.square(y_pred_fast2[-1] - y2_T) + torch.square(z_pred_fast2[-1] - z2_T)
       
    total_loss_fast2 = physics_loss_fast2 + init_loss_fast2 + boundary_loss_fast2

    return total_loss_fast2




def total_loss_func(model_fast, model_slow, model_fast2, t_tensor, tau_tensor, t2_tensor, x_real_fast, y_real_fast, z_real_fast, x_real_slow, y_real_slow, z_real_slow,x_real_fast2, y_real_fast2, z_real_fast2, x_init, y_init, z_init, x1, y1, z1, x_end, y_end, z_end, eps, weight_fast=1.0, weight_slow=1.0):
    loss_fast = loss_func_fast(model_fast, t_tensor, x_real_fast, y_real_fast, z_real_fast, x_init, y_init, z_init, x1, y1, z_exact_fast[-1], eps)
    loss_slow = loss_func_slow(model_slow, tau_tensor, x_real_slow, y_real_slow, z_real_slow, x1, y1, z1, x_exact_slow[-1],y_exact_slow[-1],z_exact_slow[-1], eps)
    loss_fast2 = loss_func_fast2(model_fast2, t2_tensor, x_real_fast2, y_real_fast2, z_real_fast2, x_end, y_end, z_end, x_exact_slow[-1],y_exact_slow[-1],z_exact_slow[-1], eps)
    total_loss = weight_fast * loss_fast + weight_slow * loss_slow + weight_fast * loss_fast2
    return total_loss






